# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [7]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2023-11-16	HuatuoGPT-II, One-stage Training for Medical Adaption of LLMs	arXiv	Junying Chen, Xidong Wang, Anningzhe Gao, Feng Jiang, Shunian Chen, Hongbo Zhang, Dingjie Song, Wenya Xie, Chuyi Kong, Jianquan Li, Xiang Wan, Haizhou Li, Benyou Wang	HuatuoGPT-II, One-stage Training for Medical Adaption of LLMs	https://arxiv.org/abs/2311.09774
2023-09-21	AceGPT, Localizing Large Language Models in Arabic	arXiv	Huang Huang, Fei Yu, Jianqing Zhu, Xuening Sun, Hao Cheng, Dingjie Song, Zhihong Chen, Abdulmohsen Alharthi, Bang An, Juncai He, Ziche Liu, Zhiyi Zhang, Junying Chen, Jianquan Li, Benyou Wang, Lian Zhang, Ruoyu Sun, Xiang Wan, Haizhou Li, Jinchao Xu	AceGPT, Localizing Large Language Models in Arabic	https://arxiv.org/abs/2309.12053
2023-08-17	CMB: A Comprehensive Medical Benchmark in Chinese	arXiv	Xidong Wang, Guiming Hardy Chen, Dingjie Song (equal contribution), Zhiyi Zhang (equal contribution), Zhihong Chen, Qingying Xiao, Fe

## Import pandas

We are using the very handy pandas library for dataframes.

In [8]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [9]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2023-11-16,"HuatuoGPT-II, One-stage Training for Medical A...",arXiv,"Junying Chen, Xidong Wang, Anningzhe Gao, Feng...","HuatuoGPT-II, One-stage Training for Medical A...",https://arxiv.org/abs/2311.09774,NaN
1,2023-09-21,"AceGPT, Localizing Large Language Models in Ar...",arXiv,"Huang Huang, Fei Yu, Jianqing Zhu, Xuening Sun...","AceGPT, Localizing Large Language Models in Ar...",https://arxiv.org/abs/2309.12053,NaN
2,2023-08-17,CMB: A Comprehensive Medical Benchmark in Chinese,arXiv,"Xidong Wang, Guiming Hardy Chen, Dingjie Song ...",CMB: A Comprehensive Medical Benchmark in Chinese,https://arxiv.org/abs/2308.08833,NaN
3,2023-10-08,Episode-based Prompt Learning for Any-shot Int...,NLPCC 2023 Oral,"Pengfei Sun, Dingjie Song (equal contribution)...",Episode-based Prompt Learning for Any-shot Int...,https://link.springer.com/chapter/10.1007/978-...,NaN
4,2022-05-31,Self-Supervised Task Augmentation for Few-Shot...,JCST 2022,"Pengfei Sun, Yawen Ouyang, Dingjie Song, Xinyu...",Self-Supervised Task Augmentation for Few-Shot...,https://link.springer.com/article/10.1007/s113...,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [10]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [11]:
import os
for row, item in publications.iterrows():

    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]

    ## YAML variables

    md = "---\ntitle: \""   + item.title + '"\n'

    md += """collection: publications"""

    md += """\npermalink: /publication/""" + html_filename

    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"

    md += "\ndate: " + str(item.pub_date)

    md += "\nvenue: '" + html_escape(item.venue) + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"

    md += "\ncitation: '" + html_escape(item.citation) + "'"

    md += "\n---"

    ## Markdown description for individual page

    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n"

    md += "\nRecommended citation: " + item.citation

    md_filename = os.path.basename(md_filename)

    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [12]:
!ls ../_publications/

2010-10-01-paper-title-number-2.md
2015-10-01-paper-title-number-3.md
2023-08-17-cmb-comprehensive-medical-benchmark-chinese.md
2308.08833.md
2309.12053.md
2311.09774.md
978-3-031-44693-1_3.md
s11390-022-2029-5.md


In [13]:
!cat ../_publications/2308.08833.md

---
title: "CMB: A Comprehensive Medical Benchmark in Chinese"
collection: publications
permalink: /publication/2023-08-17-https://arxiv.org/abs/2308.08833
excerpt: 'Xidong Wang, Guiming Hardy Chen, Dingjie Song (equal contribution), Zhiyi Zhang (equal contribution), Zhihong Chen, Qingying Xiao, Feng Jiang, Jianquan Li, Xiang Wan, Benyou Wang, Haizhou Li'
date: 2023-08-17
venue: 'arXiv'
citation: 'CMB: A Comprehensive Medical Benchmark in Chinese'
---
Xidong Wang, Guiming Hardy Chen, Dingjie Song (equal contribution), Zhiyi Zhang (equal contribution), Zhihong Chen, Qingying Xiao, Feng Jiang, Jianquan Li, Xiang Wan, Benyou Wang, Haizhou Li

Recommended citation: CMB: A Comprehensive Medical Benchmark in Chinese